# 量子模拟器

在MindQuantum中，我们可以利用量子模拟器`Simulator`来对量子线路进行模拟。这里我们声明一个两比特的`projectq`模拟器，并因此来简介模拟器的关键功能。

In [14]:
from mindquantum.simulator import Simulator

sim = Simulator('projectq', 2)     #声明一个两比特的projectq模拟器
sim                                #展示模拟器状态

projectq simulator with 2 qubits (little endian).
Current quantum state:
1¦00⟩

在MindQuantum中，我们可以在`mindquantum.simulator`模块导入模拟器。`Simulator`类可以接受三个参数：

- `backend`：所用到的模拟器名称，目前`mindquantum`支持`projectq`作为后端进行模拟。
- `n_qbuits`：模拟器所用到的比特数，也就是这里的2。
- `seed`：模拟器在运行随机性相关算法时的随机种子，默认为一个随机数，可以不用提供。

通过模拟器的输出结果我们可以发现，这是一个`projectq`的2比特模拟器，并且是little endian的。这里little endian的意思是，整个模拟器中，我们都是将比特序号小的比特放在量子态矢量的右边。接下来，输出还说明了模拟器当前所处的量子态是多少，且在模拟器初始化后，当前的量子态默认处于零态。注意，量子模拟器始终会维护一个内部的量子态，当我们作用量子门或者量子线路到模拟器上时，这个量子态会随即发生改变，而当我们只是想获取关于这个量子态的一些信息时，这个量子态则不会改变。这里就涉及到对量子模拟器的两类操作：

- 会改变量子态的操作，通常以`apply`开头，主要有如下几个
    - `apply_gate`: 作用一个量之门到模拟器上
    - `apply_circuit`: 作用一个量子线路到模拟器上
    - `apply_hamiltonian`: 将一个哈密顿量作用到模拟器上，注意，此后模拟器的量子态将不再是一个真的量子态
    - `set_qs`: 直接设置模拟器的当前量子态
    - `reset`: 重置模拟器的状态为0态
- 不会改变量子态的操作，通常以`get`开头，主要有如下几个
    - `get_qs`: 获取模拟器的当前量子态
    - `get_expectation`: 计算模拟器当前量子态关于某个观察量的期望值
    - `get_expectation_with_grad`: 跟上一个接口类似，只不过这个方法还会计算期望值关于参数化量子线路的梯度
    - `sampling`: 在当前量子态下，对给定的量子线路进行采用

下面我们简单调用一下这些接口

## 作用量子门和量子线路

In [15]:
sim = Simulator('projectq', 2)   #声明一个2比特的projectq模拟器
sim.apply_gate(H.on(0))          #作用一个Hadamard门到0号比特上
sim                              #输出量子模拟器的信息

projectq simulator with 2 qubits (little endian).
Current quantum state:
√2/2¦00⟩
√2/2¦01⟩

上面我们在量子模拟器的初态上作用了一个Hadamard门，并输出了演化过后的量子态。接下来我们生成一个参数化量子线路，并将其作用到当前的量子态上。

In [16]:
circ = Circuit()        #声明一个空的量子线路
circ += H.on(1)         #向其中添加一个hadamard门，并作用到1号比特上
circ += RY('a').on(0)   #向其中添加一个参数化的RY门，并作用到0号比特上
circ.svg()              #绘制SVG格式的量子线路图片

In [17]:
sim.apply_circuit(circ, pr={'a': 1.234})  #作用一个量子线路，当线路是一个参数化量子线路时，我们还需要提供参数值。
sim

projectq simulator with 2 qubits (little endian).
Current quantum state:
0.11851349145283657¦00⟩
0.6971044056263442¦01⟩
0.11851349145283657¦10⟩
0.6971044056263442¦11⟩

在上面的代码中，我们先生成了一个参数化量子线路`circ`，随后我们将其作用到量子模拟器上，并通过传入字典的方式，将参数`a`设置为`1.234`。最后输出量子模拟器演化出来的量子态。

## 设置并获取模拟器状态

我们使用`get_qs(ket=False)`查看当前模拟器的状态，
参数ket是一个bool类型的数，它决定了当前模拟器的状态是否以ket的形式返回，ket=False时是以numpy.ndarray形式，ket=True时是以ket形式。默认ket=False。

In [18]:
print(sim.get_qs())     #查看模拟器状态,以numpy.ndarray形式返回结果


[0.11851349+0.j 0.69710441+0.j 0.11851349+0.j 0.69710441+0.j]


In [19]:
print(sim.get_qs(True)) #查看模拟器状态,以ket形式返回结果

0.11851349145283657¦00⟩
0.6971044056263442¦01⟩
0.11851349145283657¦10⟩
0.6971044056263442¦11⟩


在实际写代码过程中，我们常常需要将模拟器指定一个初始态开始演化，这个操作可以使用`set_qs()`实现。

例如，我们希望模拟器状态为

$$
\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩
$$

第一步：我们计算出目标状态的向量形式：

$$
\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩ =\frac{\sqrt{3}}{3}\times
\left(
\begin{array}{l}
1\\
0
\end{array}
\right)
\otimes
\left(
\begin{array}{l}
1\\
0
\end{array}
\right)+
\frac{\sqrt{6}}{3}\times
\left(
\begin{array}{l}
0\\
1
\end{array}
\right)\otimes
\left(
\begin{array}{l}
0\\
1
\end{array}
\right)= \frac{\sqrt{3}}{3}\times
\left(
\begin{array}{l}
1\\
0\\
0\\
0
\end{array}
\right)+
\frac{\sqrt{6}}{3}\times
\left(
\begin{array}{l}
0\\
0\\
0\\
1
\end{array}
\right)=
\left(
\begin{array}{l}
\frac{\sqrt{3}}{3}\\
0\\
0\\
\frac{\sqrt{6}}{3}
\end{array}
\right)
$$

第二步：我们将这个向量使用`set_qs()`赋值给模拟器，让其作为模拟器的状态：

In [20]:
sim.set_qs(np.array([3**0.5/3, 0, 0, 6**0.5/3]))    #设置模拟器状态
print(sim.get_qs(True))                             #查看模拟器状态

0.5773502691896258¦00⟩
0.816496580927726¦11⟩


通过`get_qs()`查看模拟器状态可以发现，当前模拟器状态即为我们希望设置的$\frac{\sqrt{3}}{3}|00⟩+\frac{\sqrt{6}}{3}|11⟩$。

在实际编程过程中，我们常常需要多次模拟电路，通过多开模拟器的方式会导致内存占用非常大，我们可以通过现有模拟器复位的方式来复用模拟器，从而减少内存消耗。

我们使用`reset()`来复位模拟器：

In [21]:
sim.reset()         #复位模拟器
print(sim.get_qs()) #查看模拟器状态

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


可以发现，当前模拟器被复位成了初始的$1|00⟩$态，相当于一个全新的模拟器。

## 量子测量

在量子线路设计时，我们最终需要通过测量操作获得结果，我们常见的测量有Pauli测量、计算基测量、投影测量等等，此处我们以Pauli-Z测量为例展示。

在学习Pauli-Z测量之前，我们先对量子计算基测量有一个简单认识：假设有一个n个量子比特的态，我们对它执行n比特计算基测量，测量后，如果结果为$00 \cdots0$，表明该n量子比特系统的量子状态已塌缩到$|00 \cdots0⟩$态；类似地，如果测量其中一个量子比特，那么它表示的$2^n$种情况就会被排除掉一半，即在两个各占一半的空间中，测量操作将量子态投影到其中一个空间，表明该n量子比特系统的量子状态中一个子系统塌缩了。以双量子比特系统为例，假设系统处于状态

$$
\left( \begin{array}{l} \alpha_{00}\\ \alpha_{01}\\ \alpha_{10}\\ \alpha_{11} \end{array}\right)
$$

我们对其作用计算基测量，会得出00的概率为$|\alpha_{00}|^2$，01的概率为$|\alpha_{01}|^2$，10的概率为$|\alpha_{10}|^2$，11的概率为$|\alpha_{11}|^2$。如果测量结果为00，则表明该双量子比特系统的量子状态已塌缩到

$$
|00⟩=\left( \begin{array}{l} 1\\ 0\\ 0\\ 0 \end{array}\right)
$$

类似地，如果测量双量子比特量子状态的第一个量子比特，则得到0的概率为$|\alpha_{00}|^2+|\alpha_{01}|^2$，得到1的概率为$|\alpha_{10}|^2+|\alpha_{11}|^2$。

通过对计算基测量的学习，我们可以直观认识到，在多量子比特态的其中一个比特上做测量，本质是将量子态投影到一个子空间中。为了简洁的区分出这两个子空间，我们利用线性代数知识知道，可以通过恰好有两个唯一特征值的矩阵来描述两个正交子空间。

接下来我们学习Pauli-Z测量，我们考虑Z算子：

$$
Z=
\left(
    \begin{array}{l}
    1&0\\
    0&-1
    \end{array}
\right)
$$

可以看出，Z有两个特征值+1，-1，对应的特征向量分别为：|0⟩和|1⟩。因此，如果在量子比特上作用Z门后测量结果为0（对应状态为|0⟩），我们可得出该量子比特的状态是在Z算子的+1特征子空间$V_{+1}$中，相似地，如果测量结果为1可得出该量子比特的状态是在Z算子的-1特征子空间$V_{-1}$中。这即为Pauli-Z测量，仔细阅读的读者可以发现，这其实就是计算基测量。

最后，我们认识一下投影测量(projective measuremen)，投影测量由一个可观测量（observable）$A$来描述，可观测量$A$是一个待观测系统的状态空间上的自伴算子，满足$A=A^{\dagger}$。可观测量A可以写成谱分解的形式$A=\Sigma_{i}\lambda_{i}P_i$.

这里的$P_i$为在$A$的特征值$\lambda_i$对应特征空间上的投影。

在对状态$|\psi⟩$测量之后，得到结果$i$的概率为$p_i=p(\lambda_i)=⟨\psi|P_i|\psi⟩$.

投影测量一个重要的特征就是很容易计算投影测量的平均值$E(A)$。

$$
\begin{align*}
    E(A) &=\Sigma_i \lambda_i p_i\\
         &=\Sigma_i \lambda_i⟨\psi|P_i|\psi⟩\\
         &=⟨\psi|(\Sigma_i\lambda_i P_i)|\psi⟩\\
         &=⟨\psi|A|\psi⟩
\end{align*}
$$

投影测量的好处是，当测量算子具有酉变换性质时，投影测量和一般测量等价。

`get_expectation(hamiltonian)`是MindQuantum提供的投影测量API，它可以计算出模拟器当前量子态关于某个可观测量$H$的期望值：$E=⟨\psi|H|\psi⟩=0 \times p(0) + 1 \times p(1)$，其中$p(0),p(1)$分别表示测量结果为0，1的概率。我们常常称$H$为哈密顿量（Hamitonian），利用这个API，我们可以通过构造$H$来获得相应的测量结果。

例如，我们希望对处于$\frac{\sqrt{2}}{2}|00⟩+\frac{\sqrt{2}}{2}|11⟩$态的系统上的q1比特上作用一个Pauli-Z测量，首先我们将模拟器置位：

In [22]:
sim.set_qs(np.array([2**0.5 / 2, 0, 0, 2**0.5 / 2]))  # 设置模拟器状态
print(sim.get_qs())


[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


然后我们构造出在q1上做Pauli-Z测量对应的哈密顿量hams:

In [23]:
from mindquantum import Hamiltonian                     # 引入哈密顿量定义模块
from mindquantum.core.operators import QubitOperator    # 引入稀疏算子定义模块

hams = Hamiltonian(QubitOperator('Z1'))                 # 构建在q1上作Pauli-Z测量的哈密顿量

为了直观认识到测量操作，我们先手动计算出模拟器当前量子态在q1上做Pauli-Z测量的期望值：

$$
\begin{align*}
E&=⟨\psi|H|\psi⟩\\&=
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}& 0& 0& \frac{\sqrt{2}}{2}
    \end{array}
\right) \times
(Z \otimes I) \times
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}\\
    0\\
    0\\
    \frac{\sqrt{2}}{2}
    \end{array}
\right) \\&=
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}& 0& 0& \frac{\sqrt{2}}{2}
    \end{array}
\right) \times
\left(
    \begin{array}{l}
    1&0\\
    0&-1\\
    \end{array}
\right) \otimes
\left(
\begin{array}{l}
    1&0\\
    0&1\\
\end{array}
\right)
\times
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}\\
    0\\
    0\\
    \frac{\sqrt{2}}{2}
    \end{array}
\right) \\&=
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}& 0& 0& \frac{\sqrt{2}}{2}
    \end{array}
\right) \times
\left(
    \begin{array}{l}
    1&0&0&0\\
    0&1&0&0\\
    0&0&-1&0\\
    0&0&0&-1
    \end{array}
\right)
\times
\left(
    \begin{array}{l}
    \frac{\sqrt{2}}{2}\\
    0\\
    0\\
    \frac{\sqrt{2}}{2}
    \end{array}
\right) \\&=
0
\end{align*}
$$

In [24]:
sim.get_expectation(hams)  # 计算出模拟器当前量子态关于hams的期望值

0j

可以看到，手动计算和使用`get_expectation(hamiltonian)`计算出的结果相同，符合预期。

## 量子线路采样

线路采样是指对量子线路执行多次模拟测量，统计测量出各种结果出现的频次。采样不会改变量子线路中的状态。

`sampling(circuit, pr=None, shots=1, seed=None)`是`MindQuantum`中提供的对模拟器进行线路采样方法，它接受四个参数：

- `circuit (Circuit)`：希望进行采样的量子线路，注意，该线路中必须包含至少一个测量操作（即采样点）。
- `pr (Union[None, dict, ParameterResolver])`：parameter resolver，当 `circuit`是含参线路时，需要给出参数的值。
- `shots (int)`：采样的次数，默认为1。
- `seed`：采样时的随机种子，默认为一个随机数，可以不用提供。

In [25]:
# 不含参线路采样：
from mindquantum import Measure # 引入测量门

circ = Circuit()                # 初始化量子线路
circ += H.on(0)                 # H门作用在第0位量子比特
circ += X.on(1, 0)              # X门作用在第1位量子比特且受第0位量子比特控制
circ += Measure('q0').on(0)      # 在0号量子比特作用一个测量，并将该测量命名为'q0'
circ += Measure('q1').on(1)     # 在1号量子比特作用一个测量，并将该测量命名为'q1'
circ.svg()                      # 绘制SVG格式的量子线路图片


In [26]:
sim.reset()
result = sim.sampling(circ, shots=1000) # 对上面定义的线路采样1000次
result


shots: 1000
Keys: q1 q0│0.00   0.125       0.251       0.376       0.501       0.626
───────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
         00│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
           │
         11│▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
           │
{'00': 501, '11': 499}

我们可以看到，采样结果中'00'出现了499次，'11'出现了501次。我们搭建的线路实际上制备出了一个贝尔态$\frac{\sqrt{2}}{2}|00⟩+\frac{\sqrt{2}}{2}|11⟩$。直观上，我们可以看到对该状态进行测量得到'00'的概率为$\frac{1}{2}$,得到'11'的概率为$\frac{1}{2}$，采样结果符合概率，细微的误差是由模拟器噪声导致。

In [27]:
# 含参线路采样：
para_circ = Circuit()               # 初始化量子线路
para_circ += H.on(0)                # H门作用在第0位量子比特
para_circ += X.on(1, 0)             # X门作用在第1位量子比特且受第0位量子比特控制
para_circ += RY('theta').on(1)      # RY(theta)门作用在第2位量子比特
para_circ += Measure('0').on(0)     # 在0号量子比特作用一个测量，并将该测量命名为'q0'
para_circ += Measure('q1').on(1)    # 在1号量子比特作用一个测量，并将该测量命名为'q1'
para_circ.svg()                     # 绘制SVG格式的量子线路图片

In [28]:
sim.reset()
result = sim.sampling(para_circ, {'theta': 0}, shots=1000)  # 将上面定义的线路参数'theta'赋值为0采样1000次
result

shots: 1000
Keys: q1 0│0.00   0.128       0.255       0.383       0.511       0.639
──────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
        00│▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒
          │
        11│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
          │
{'00': 489, '11': 511}

In [29]:
result.svg()

我们可以看到，采样结果中'00'出现了489次，'11'出现了511次。事实上把RY门参数赋值为0，它即为我们熟悉的I门，相当于不对线路做任何操作，因此该采样线路与上面不含参线路本质是同一个，可以观察到二次采样结果几乎相同，符合预期结果。